In [ ]:
#Extracting text from PDF Files

import PyPDF2
import re
import pandas as pd
!pip install transformers --quiet

In [ ]:
# Load and extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() or ""
    return text

In [ ]:
# Preprocessing/Cleaning

def preprocess_text(text):
    text = re.sub(r"\n+", " ", text)
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    return text.lower()

In [ ]:
# Split resumes based on 'PROFESSIONAL SUMMARY' section

def split_resumes_by_section(text):
    sections = text.split("PROFESSIONAL SUMMARY")
    resumes = []
    for i, section in enumerate(sections[1:], start=1):  # skip preamble
        # Use preceding line or first line after split as name
        preceding = sections[i-1].strip().splitlines()
        name_line = preceding[-1].strip() if preceding else f"Candidate {i}"
        name = re.findall(r"[A-Z][a-z]+ [A-Z][a-z]+", name_line)
        name = name[0] if name else f"Candidate {i}"

        cleaned_resume = preprocess_text(section)
        resumes.append({"Name": name, "Resume_Text": cleaned_resume})
    return pd.DataFrame(resumes)

In [ ]:
from google.colab import files

uploaded = files.upload()  # This will open a file dialog in Colab

pdf_path = "All_Resumes.pdf"
raw_text = extract_text_from_pdf(pdf_path)
resume_df = split_resumes_by_section(raw_text)

# # Save or inspect
# resume_df.to_csv("classified_resumes.csv", index=False)
# resume_df.head()

Saving All_Resumes.pdf to All_Resumes.pdf


In [ ]:
from transformers import pipeline

# Load zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Define potential job categories (you can expand or customize this list)
candidate_labels = ["Data Science", "Software Development", "HR", "Legal", "Marketing", "Finance", "Operations", "Sales", "Design", "Education", "Others"]

# Function to classify a single resume dynamically
def dynamic_classify_resume(text):
    result = classifier(text, candidate_labels, multi_label=False)
    return result["labels"][0]  # best prediction

# Apply to DataFrame
resume_df["Predicted_Job_Category"] = resume_df["Resume_Text"].apply(dynamic_classify_resume)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [10]:
resume_df.to_csv("classified_resumes_dynamic.csv", index=False)
resume_df.head()

,Name,Resume_Text,Predicted_Job_Category
0,Alice Johnson,experienced specialist with extensive backgro...,Data Science
1,Machine Learning,innovative thinker with a passion for deliver...,Software Development
2,Ethan Martin,highly motivated professional with a track re...,HR
3,Employee Relations,experienced specialist with extensive backgro...,HR
4,Employee Relations,highly motivated professional with a track re...,HR
